<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#téléchargement-en-parallèle" data-toc-modified-id="téléchargement-en-parallèle-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>téléchargement en parallèle</a></span></li></ul></div>

In [6]:
import pandas as pd
import requests

In [7]:
df = pd.read_excel('repertoires.xlsx')
df.dropna(subset=['nom_court'], inplace=True)
df.head()

,nom_court,annee,remarques,lien_source,source_ID,vue_debut,page_debut,vue_fin,page_fin,page_total,pages,nom_fichier
0,Annuaire-general-du-commerce,1839.0,NaN,https://gallica.bnf.fr/ark:/12148/bpt6k63243601,FRBNF32698036,123.0,103.0,399.0,379,1457,277.0,bpt6k63243601.Alto.html
1,Annuaire-general-du-commerce,1841.0,"document en 2 parties sur Gallica, seule la pr...",https://gallica.bnf.fr/ark:/12148/bpt6k62931221,FRBNF32698036,151.0,72.0,423.0,344,938,273.0,bpt6k62931221.Alto.html
2,Annuaire-general-du-commerce,1842.0,"document en 2 parties sur Gallica, seule la pr...",https://gallica.bnf.fr/ark:/12148/bpt6k6286466w,FRBNF32698036,189.0,91.0,489.0,391,939,301.0,bpt6k6286466w.Alto.html
3,Annuaire-general-du-commerce,1843.0,"document en 2 parties sur Gallica, seule la pr...",https://gallica.bnf.fr/ark:/12148/bpt6k6393838j,FRBNF32698036,219.0,107.0,482.0,378,969,264.0,bpt6k6393838j.Alto.html
4,Annuaire-general-du-commerce,1844.0,La dernière page comprend le début d'une autre...,https://gallica.bnf.fr/ark:/12148/bpt6k6331310g,FRBNF32698036,216.0,103.0,483.0,372,977,268.0,bpt6k6331310g.Alto.html


In [8]:
df['pages'].sum()

27576.0

In [9]:
df['ids'] = df['lien_source'].str.replace('https://gallica.bnf.fr/ark:/12148/', '')
df['vue_debut'] = df['vue_debut'].astype(int)
df['vue_fin'] = df['vue_fin'].astype(int)

In [40]:
urls = []
for id, debut, fin in df[['ids', 'vue_debut', 'vue_fin']].values:
    for i in range(debut, fin+1):
        urls.append(("https://gallica.bnf.fr/RequestDigitalElement?O=%s&E=ALTO&Deb=%d"%(id, i), "ocr_alto/%s_%d_alto.xml"%(id, i)))

In [41]:
utils_alto.download(urls[0])

In [42]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
%autoreload 2

In [46]:
from multiprocessing import Pool
import utils_alto
from tqdm import tqdm_notebook as tqdm
import os

pbar = tqdm(total=len(urls))

def update(i):
    pbar.update()
    
if __name__ == '__main__':
    pool = Pool(32)
    for item in urls:
        _ , file = item
        pool.apply_async(utils_alto.download, args=(item,), callback=update)
    pool.close()
    pool.join()
    pbar.close()

In [29]:
df['texteBrut'] = df['lien_source'] + '/f.' + df['vue_debut'].astype(str) + 'n' + df['pages'].astype(str) + '.texteBrut'
df['texteBrut'].values

array(['https://gallica.bnf.fr/ark:/12148/bpt6k63243601/f.123n276.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k62931221/f.151n272.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k6286466w/f.189n300.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k6393838j/f.219n263.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k6331310g/f.216n267.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k6292987t/f.353n276.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k62906378/f.288n280.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k6391515w/f.319n297.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k6315927h/f.349n325.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k6319106t/f.324n316.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k6315985z/f.82n322.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k63959929/f.82n342.texteBrut',
       'https://gallica.bnf.fr/ark:/12148/bpt6k63197984/f.57n360.t

In [27]:
url = df['texteBrut'].iloc[0]

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

res = requests.get(url, headers=headers)


In [28]:
res.ok

False

In [3]:
df['lien_source'] = df['lien_source'].str.replace('https://gallica.bnf.fr/', '')
df.head(5)

,nom_court,annee,remarques,lien_source,source_ID,vue_debut,page_debut,vue_fin,page_fin,page_total,pages
0,Annuaire-general-du-commerce,1839,NaN,ark:/12148/bpt6k63243601,,123,103,399,379,1457,276
1,Annuaire-general-du-commerce,1841,"document en 2 parties sur Gallica, seule la pr...",ark:/12148/bpt6k62931221,,151,72,423,344,938,272
2,Annuaire-general-du-commerce,1842,"document en 2 parties sur Gallica, seule la pr...",ark:/12148/bpt6k6286466w,,189,91,489,391,939,300
3,Annuaire-general-du-commerce,1843,"document en 2 parties sur Gallica, seule la pr...",ark:/12148/bpt6k6393838j,,219,107,482,378,969,271
4,Annuaire-general-du-commerce,1844,La dernière page comprend le début d'une autre...,ark:/12148/bpt6k6331310g,,216,103,483,372,977,269


In [4]:
df['vue_debut'] = [int(i) for i in df['vue_debut']]
df['vue_fin'] = [int(i) for i in df['vue_fin']]
df['pages'] = [int(i) for i in df['pages']]


In [5]:
list_url = []
for i, rows in df.iterrows():
    for j in range(rows.pages+1):
        f = str(rows.vue_debut + j)
        file_name = 'data/' + rows.nom_court + '_' + str(rows.annee) + '_' + str(j+1) + '.jpg'
        url = 'https://gallica.bnf.fr/iiif/' + rows.lien_source + '/f' + f + '/full/full/0/' + 'native.jpg'
        list_url.append((url, file_name))
list_url

[('https://gallica.bnf.fr/iiif/ark:/12148/bpt6k63243601/f123/full/full/0/native.jpg',
  'data/Annuaire-general-du-commerce_1839_1.jpg'),
 ('https://gallica.bnf.fr/iiif/ark:/12148/bpt6k63243601/f124/full/full/0/native.jpg',
  'data/Annuaire-general-du-commerce_1839_2.jpg'),
 ('https://gallica.bnf.fr/iiif/ark:/12148/bpt6k63243601/f125/full/full/0/native.jpg',
  'data/Annuaire-general-du-commerce_1839_3.jpg'),
 ('https://gallica.bnf.fr/iiif/ark:/12148/bpt6k63243601/f126/full/full/0/native.jpg',
  'data/Annuaire-general-du-commerce_1839_4.jpg'),
 ('https://gallica.bnf.fr/iiif/ark:/12148/bpt6k63243601/f127/full/full/0/native.jpg',
  'data/Annuaire-general-du-commerce_1839_5.jpg'),
 ('https://gallica.bnf.fr/iiif/ark:/12148/bpt6k63243601/f128/full/full/0/native.jpg',
  'data/Annuaire-general-du-commerce_1839_6.jpg'),
 ('https://gallica.bnf.fr/iiif/ark:/12148/bpt6k63243601/f129/full/full/0/native.jpg',
  'data/Annuaire-general-du-commerce_1839_7.jpg'),
 ('https://gallica.bnf.fr/iiif/ark:/12148

In [6]:
len(list_url)

10946

### téléchargement en parallèle

In [18]:
'''for item in tqdm(list_url):
    download(item)'''

In [7]:
'''from multiprocessing import Pool
import utils
from tqdm import tqdm_notebook as tqdm

pbar = tqdm(total=len(list_url))

def update(i):
    pbar.update()
    
if __name__ == '__main__':
    pool = Pool(32)
    for item in list_url:
        pool.apply_async(utils.download, args=(item,), callback=update)
    pool.close()
    pool.join()
    pbar.close()'''